In [1]:
from pyspark.sql import SparkSession

In [2]:
sparksession=SparkSession.builder.appName('sqlconnection').getOrCreate()

In [3]:
spark = SparkSession.builder.config("spark.jars", "/usr/share/java/mysql-connector-java-8.0.22.jar") \
    .master("local").appName("PySpark_MySQL_test").getOrCreate()

In [5]:
config = {
    'host': '127.0.0.1',
    'port': 3307,
    'user': 'root',
    'password': 'root',
    'database': 'capstone',
    'test_table': 'news_data'
}
db_user = config.get('user')
db_pwd = config.get('password')
db_host = config.get('host')
db_port = config.get('port')
db_name = config.get('database')# specify connection string
db_table=config.get('test_table')

In [3]:
"""import sqlalchemy as db
import pymysql as pymysql

# specify database configurations


connection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'# connect to database
engine = db.create_engine(connection_str)
connection = engine.connect()# pull metadata of a table
metadata = db.MetaData(bind=engine)
metadata.reflect(only=[db_table])

test_table = metadata.tables[db_table]

test_table
"""

"import sqlalchemy as db\nimport pymysql as pymysql\n\n# specify database configurations\n\n\nconnection_str = f'mysql+pymysql://{db_user}:{db_pwd}@{db_host}:{db_port}/{db_name}'# connect to database\nengine = db.create_engine(connection_str)\nconnection = engine.connect()# pull metadata of a table\nmetadata = db.MetaData(bind=engine)\nmetadata.reflect(only=[db_table])\n\ntest_table = metadata.tables[db_table]\n\ntest_table\n"

In [6]:
import mysql.connector as mysql
db = mysql.connect(
    host = db_host,
    user = db_user,
    passwd = db_pwd,
    database = db_name,
    port=db_port
)

In [7]:
connn=db.cursor()

In [8]:
query='select count(*) from news_data'
res=connn.execute(query)

In [9]:
print(res)

None


In [10]:
res=connn.fetchall()

In [11]:
print(res)

[(49028,)]


In [12]:
query='select * from news_data'
connn.execute(query)
records=connn.fetchall()


In [64]:
len(records[0]

6

In [13]:
from datetime import datetime

In [14]:
ids=[]
title=[]
createddate=[]
summary=[]
topic=[]
urls=[]
for rec in records:
    ids.append(rec[0])
    title.append(rec[1])
    createddate.append(str(rec[2]))
    summary.append(rec[3])
    topic.append(rec[4])
    urls.append(rec[5])

In [75]:
records

list

In [15]:
data={'id':ids,
'title':title,
'createddate':createddate,
'summary':summary,
'topic':topic,
'source_url':urls}

In [16]:
from pyspark.sql import SparkSession
import pandas as pd

In [17]:
sparksession=SparkSession.builder.appName('preprocessor').getOrCreate()


In [18]:
df=pd.DataFrame(data)

In [26]:
df.to_csv('../data/data.csv',index=False,header=True,sep='|')

In [ ]:
df.to

In [19]:
rdd=sparksession.createDataFrame(df)

In [20]:
rdd.take(1)

[Row(id=1, title='Football Canada, artist Grant produce identity for football in Indigenous communities', createddate='2021-09-29', summary="Football Canada and Indigenous artist Kolten Khasalus Grant have collaborated to produce a national identity for football in Indigenous communities across the country. The Indigenous Football Canada logo will be available on merchandise for Football Weekend in Canada on Oct. 15. Net proceeds will be directed toward the development and support of Indigenous football. 'Football Canada's commitment to diversity is always evolving,' Football Canada president Jim Mullin said in a statement. 'This stu", topic='sport', source_url='theglobeandmail.com')]

In [29]:
rdd=rdd.dropna(how='any')

In [31]:
rdd_collect=rdd.collect()

In [76]:
len(rdd_collect)

48329

In [77]:
rdd_collect[:5]

[Row(id=1, title='Football Canada, artist Grant produce identity for football in Indigenous communities', createddate='2021-09-29', summary="Football Canada and Indigenous artist Kolten Khasalus Grant have collaborated to produce a national identity for football in Indigenous communities across the country. The Indigenous Football Canada logo will be available on merchandise for Football Weekend in Canada on Oct. 15. Net proceeds will be directed toward the development and support of Indigenous football. 'Football Canada's commitment to diversity is always evolving,' Football Canada president Jim Mullin said in a statement. 'This stu", topic='sport', source_url='theglobeandmail.com'),
 Row(id=2, title='UCLA Stanford Football', createddate='2021-10-01', summary='Stanford quarterback Tanner McKee (No. 18) throws a touchdown pass against UCLA during the second half on Saturday, Sept. 25, 2021, in Stanford, Calif.', topic='news', source_url='apnews.com'),
 Row(id=3, title="'It's natural as

In [34]:
import nltk
from string import punctuation
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [32]:
corpus=[]
topic=[]
for row in rdd_collect:
    text=row['title']+' '+row['summary']
    corpus.append(text)
    topic.append(row['topic'])
    

In [33]:
len(corpus),len(topic)

(48329, 48329)

In [45]:
updated_corpus=[]
stemmer=PorterStemmer()
for row in corpus:
    sents=' '.join([sent for sent in sent_tokenize(row)])
    words=[word for word in word_tokenize(sents) if not word in stopwords.words('english')]
    words=[word for word in words if len(word)>1]
    words=[word for word in words if not word in punctuation]
    #tagged_words=[nltk.pos_tag(word) for word in words]
    words=' '.join([stemmer.stem(word) for word in words])
    updated_corpus.append(words)
    break;





In [46]:
updated_corpus

["footbal canada artist grant produc ident footbal indigen commun footbal canada indigen artist kolten khasalu grant collabor produc nation ident footbal indigen commun across countri the indigen footbal canada logo avail merchandis footbal weekend canada oct. 15 net proce direct toward develop support indigen footbal 'footbal canada 's commit divers alway evolv footbal canada presid jim mullin said statement 'thi stu"]

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [41]:
c=[word for word in a if word not in ['!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~']]

In [43]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ravikumar/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
topic

['sport']

In [47]:
import json

In [60]:
with open('../preprocessor/db_config.json','r') as f:
    a=json.load(f)[0]

In [61]:
print(a)

{'host': '127.0.0.1', 'port': 3307, 'user': 'root', 'password': 'root', 'database': 'capstone', 'test_table': 'news_data'}


In [62]:
a['host']

'127.0.0.1'

In [65]:
schema=['id','title','createddate','summary','topic','sourceurl']

In [66]:
import pickle

In [69]:
pickle.dump(schema,open('../data/schema.pkl','wb'))

In [71]:
x=pickle.load(open('../data/schema.pkl','rb'))

In [73]:
type(x)

list

In [78]:
import os

In [80]:
os.p

AttributeError: module 'os' has no attribute 'abspath'

In [81]:
k='ravkasd8*'

In [82]:
import re

In [84]:
re.sub('[^a-zA-Z0-9.]+', '',k)

'ravkasd8'